In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from utils import *

In [2]:
df = pd.read_csv("./DatosSinapsisArtificial/TrozoC_LP_VD_bin.csv")[["LP_bin", "VD_bin"]]

In [3]:
df.head()

,LP_bin,VD_bin
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [4]:
%%time
mutual_info(df, ['LP_bin', 'VD_bin'], 1)

CPU times: user 21.3 s, sys: 1.38 s, total: 22.6 s
Wall time: 22.6 s


2.0614755821476982e-06

In [5]:
%%time
mutual_info(df, ['LP_bin', 'VD_bin'], 2)

CPU times: user 32min 44s, sys: 6.31 s, total: 32min 51s
Wall time: 32min 52s


7.4095807517035595e-06